In [1]:
import os
import pickle
%matplotlib inline
import pandas as pd
import seaborn as sns
import re
from collections import defaultdict, Counter
from tqdm import tqdm

In [2]:
edges = pd.read_csv('edges_filtered.csv')

In [3]:
nodes = pd.read_csv("nodes_filtered.csv")

In [4]:
abv = pd.read_csv("abv.csv")
name_abv = dict(zip(abv.full_name, abv.abv))
abv_name = dict(zip(abv.abv, abv.full_name))

In [5]:
edges.head()

,START_ID,END_ID,TYPE,pmids,n_pmids,NEG,DOMAIN,PRED,RANGE
0,C0021769,C1182654,CDatA,1597294,1,False,CD,at,A
1,C0001271,C0007635,CDatA,6684662;11037792,2,False,CD,at,A
2,C0003339,C0014467,CDatA,11857592,1,False,CD,at,A
3,C0028612,C0085080,CDatA,1985199,1,False,CD,at,A
4,C0059249,C0431085,CDatA,12739069,1,False,CD,at,A


In [6]:
nodes.head()

,ID,label,TYPE
0,C0038195,State Medicine,Activities & Behaviors
1,C1290952,Taking medication,Activities & Behaviors
2,C0680552,legalization,Activities & Behaviors
3,C0085092,Parenting behavior,Activities & Behaviors
4,C1096771,Murderer,Activities & Behaviors


In [7]:
nodes.TYPE.value_counts()

Chemicals & Drugs              95803
Living Beings                  47285
Disorders                      39973
Genes & Molecular Sequences    20849
Anatomy                        15509
Procedures                     14025
Physiology                      8222
Concepts & Ideas                4291
Devices                         3873
Objects                         1437
Activities & Behaviors          1386
Phenomena                       1162
Organizations                    892
Geographic Areas                 494
Occupations                      457
Name: TYPE, dtype: int64

In [8]:
# number of edges by node type
Counter(list(edges.DOMAIN) + list(edges.DOMAIN)).most_common()

[('CD', 15855098),
 ('G', 6276644),
 ('A', 5433270),
 ('DO', 3069628),
 ('PR', 1967574),
 ('LB', 1618726),
 ('PS', 731956),
 ('DV', 203056),
 ('PH', 104548),
 ('OR', 82326),
 ('AB', 80390),
 ('OB', 78912),
 ('CI', 69452),
 ('GA', 3998),
 ('OC', 2870)]

In [9]:
def print_most_common(node_type):
    x = edges[edges.DOMAIN == node_type].START_ID
    y = edges[edges.RANGE == node_type].END_ID
    c = [x[0] for x in Counter(list(x) + list(y)).most_common(10)]
    print(nodes.loc[c])

In [10]:
# [print_most_common(x) for x in set(edges.domain)]

In [11]:
# these are not useful or would be very difficult to normalize
remove_nodes = ['Concepts & Ideas', 'Organizations', 'Geographic Areas', 'Occupations', 'Devices',
               'Procedures']
remove_nodes = [name_abv[x] for x in remove_nodes]
remove_nodes

['CI', 'OR', 'GA', 'OC', 'DV', 'PR']

In [12]:
def drop_nodes(edges, remove_nodes):
    idx = edges[edges.DOMAIN.isin(remove_nodes) | edges.RANGE.isin(remove_nodes)].index
    edges.drop(idx, inplace=True)
print(len(edges))
drop_nodes(edges, remove_nodes)
print(len(edges))

17789224
16421859


In [13]:
nodes = nodes[nodes.ID.isin(edges.START_ID) | nodes.ID.isin(edges.END_ID)]

In [14]:
nodes.TYPE.value_counts()

Chemicals & Drugs              95679
Living Beings                  47138
Disorders                      38964
Genes & Molecular Sequences    20848
Anatomy                        15305
Physiology                      8199
Objects                         1437
Phenomena                       1151
Activities & Behaviors          1030
Name: TYPE, dtype: int64

In [15]:
nodes.to_csv("nodes_filtered2.csv", index=False)

In [16]:
edges.to_csv("edges_filtered2.csv", index=False)